In [19]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
import string

In [20]:
def cleanTotal(s):
    # function to clean total funding column so that values are single integers
    letters = string.ascii_lowercase
    punct = string.punctuation
    punct = punct.replace('$','')
    new = ''
    for i in range(len(s)):
        if s[i].lower() not in letters and s[i] not in punct and s[i] != ' ':
            new += s[i]
    if len(new) == 0:
        return 0
    new = new.replace('$','+')
    if new[0] == '+':
        new = new[1:]
    values = new.split('+')
    total = 0
    for i in range(len(values)):
        total += int(values[i])
    return total

In [21]:
# import merged csv to DataFrame
mergedData = pd.read_csv("merged_eviction_data.csv")

In [22]:
# dropping columns not pertaining to amount of funding allocated and funding source
fundingSource = mergedData.drop(columns = ['Administering Agency','Max $ Assistance per household (total)',
       'Household Assistance Details (# of months, max per month, etc)',
       'Area Median Income Elgibility', 'Program Timeline', 'Contact info',
       'Link to application', 'Notes', 'Unnamed: 11', 'Residents 2020-10-24',
       'Count 2020-10-24', 'Residents 2020-10-31', 'Count 2020-10-31',
       'Residents 2020-11-07', 'Count 2020-11-07', 'Residents 2020-11-14',
       'Count 2020-11-14', 'Residents 2020-11-21', 'Count 2020-11-21',
       'Residents 2020-11-28', 'Count 2020-11-28', 'Residents 2020-12-05',
       'Count 2020-12-05', 'Residents 2020-12-12', 'Count 2020-12-12',
       'Residents 2020-12-19', 'Count 2020-12-19', 'Residents 2020-12-26',
       'Count 2020-12-26', 'Residents 2021-01-02', 'Count 2021-01-02',
       'Residents 2021-01-09', 'Count 2021-01-09', 'Residents 2021-01-16',
       'Count 2021-01-16', 'Residents 2021-01-23', 'Count 2021-01-23',
       'Residents 2021-01-30', 'Count 2021-01-30', 'Residents 2021-02-06',
       'Count 2021-02-06', 'Residents 2021-02-13', 'Count 2021-02-13',
       'Residents 2021-02-20', 'Count 2021-02-20', 'Residents 2021-02-27',
       'Count 2021-02-27'])

In [23]:
# cleaning funding source column for first level of standardization
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(', and ', '/')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(', ', '/')

fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust and CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust, CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CPA/Trust', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust and CDBG', 'Trust/CDBG')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Not specified', 'NotSpecified')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CDBG-CV ', 'CDBG-CV')


fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CARES Act funds', 'CARES Act')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CARES Act ', 'CARES Act')

In [24]:
x = fundingSource[['Funding Source']].values.flatten().tolist()
sorted(x)

['$100,000 from Brookline Housing Trust and $375,000 from CDBG and $725,000 from Private Fundraising',
 'Affordable Housing Trust',
 'CARES Act',
 'CARES Act',
 'CARES Act',
 'CARES Act',
 'CDBG',
 'CDBG',
 'CDBG-CV',
 'CDBG-CV',
 'CDBG-CV/CPA',
 'CDBG-CV/CPA',
 'CDBG-CV/CPA/regular CDBG/donations',
 'CDBG-CV/HOME/One Haverhill Fund through United Way',
 'CDBG/HOME-TBRA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA',
 'CPA/city funds',
 'City funds',
 'City funds allocated by Mayor',
 'Donations',
 'Donations ',
 'Donations - MA COVID Relief Fund/CDBG',
 "Donations were used for Mayor's Distaster Relief Funds/",
 'Donations/various town allocations',
 'ESG-CV',
 'ESG-CV',
 'Existing Lexington Emergency Assistance Fund (LEAF) recieved additional donations in response to COVID-19 crisis',
 'For emergency fund/the city is using CDBG and CARES act funds.',
 'Foundation funding',
 'HOME',
 'HOME',
 'HOME/CDBG',
 'HOME

In [41]:
## playing around with methods of doing all of the sorting all at once, so far I'm stuck where when I merge for some reason it merges the first row and then after that it creates new rows for everything else. 
test = pd.DataFrame()
test['Municipality'] = fundingSource['Municipality']
# test['Amount Allocated'] = fundingSource['Amount Allocated']
def fundingSplit(r):
    global test
    res = dict([('Municipality',r[0]),('Total',cleanTotal(r[1])),('Amount Allocated',r[1])])
    sources = r[2].split('/')
    for i in range(len(sources)):
        if 'CARES' in sources[i]:
            res['CARES'] = r[1]
        elif 'Trust' in sources[i]:
            res['Trust'] = r[1]
        elif 'CPA' in sources[i]:
            res['CPA'] = r[1]
        elif 'CDBG-CV' in sources[i]:
            res['CDBG-CV'] = r[1]
        elif 'CDBG' in sources[i]:
            res['CDBG'] = r[1]
        elif 'HOME' in sources[i]:
            res['HOME'] = r[1]
        elif 'HOME-TBRA' in sources[i]:
            res['HOME-TBRA'] = r[1]
        elif 'city funds' in sources[i].lower():
            res['City Funds'] = r[1]
        elif 'donations' in sources[i].lower():
            res['Donations'] = r[1]
        else:
            res['Other'] = sources[i]
    insert = pd.DataFrame.from_dict(res,orient='index').transpose()
    cols = []
    for c in range(len(list(insert.columns))):
        if list(insert.columns)[c] in list(test.columns):
            cols+= [list(insert.columns)[c]]
    test = pd.merge(test,insert,how='outer',on=cols)
    # print(res)

result = [fundingSplit(row) for row in zip(fundingSource['Municipality'], fundingSource['Amount Allocated'], fundingSource['Funding Source'])]

test.to_csv('test.csv')

In [42]:
test

,Municipality,Total,Amount Allocated,CARES,Other,Trust,CDBG-CV,CPA,CDBG,Donations,HOME,City Funds
0,Acton,55000.0,"$55,000","$55,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amesbury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amherst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barnstable County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
149,Weymouth,239773.0,"239,773",NaN,NaN,NaN,"239,773",NaN,NaN,NaN,NaN,NaN
150,Williamstown,95700.0,"$20,700 Trust, $75,000 CPA",NaN,NaN,"$20,700 Trust, $75,000 CPA",NaN,"$20,700 Trust, $75,000 CPA",NaN,NaN,NaN,NaN
151,Yarmouth,108000.0,"$108,000",NaN,NaN,"$108,000",NaN,"$108,000",NaN,NaN,NaN,NaN
152,"South Berkshire Consortium, see notes*",220000.0,"$220,000",NaN,Foundation funding,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# loading this initial dataframe to a csv for error checking
fundingSource.to_csv('fundingsources.csv')

In [28]:
## fixing some of the municipalities that have multiple sources

## ARLINGTON
values = fundingSource.loc[fundingSource['Municipality'] == 'Arlington',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()[1].split(', ')
a = dict([('Municipality','Arlington'),('CPA',values[1]),('CDBG-CV',values[0]),('Donations',values[2]),('Housing Corporation',values[3])])
arlington = pd.DataFrame.from_dict(a,orient='index').transpose()

## BROOKLINE
values = fundingSource.loc[fundingSource['Municipality'] == 'Brookline',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
values
b = dict([('Municipality','Brookline'),('CDBG-CV',values[2].split('and')[1]),('Town Sources', values[1]),('Housing Trust',values[2].split('and')[0]), ('Private Fundraising', values[2].split('and')[2])])
brookline = pd.DataFrame.from_dict(b,orient='index').transpose()


## BOSTON
values = fundingSource.loc[fundingSource['Municipality'] == 'Boston',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
bos = dict([('Municipality','Boston'),('Rental Relief Lottery', values[2].split(' - ')[0]),('Federal Stimulus Funds',values[2].split(' - ')[1])])
boston = pd.DataFrame.from_dict(bos,orient='index').transpose()

## CAMBRIDGE
values = fundingSource.loc[fundingSource['Municipality'] == 'Cambridge',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
cam = dict([('Municipality','Cambridge'), ('City Disaster Relief Funds/Donations', values[1])])
cambridge = pd.DataFrame.from_dict(cam,orient='index').transpose()
cambridge

## FRAMINGHAM
'For emergency fund/the city is using CDBG and CARES act funds.'
values = fundingSource.loc[fundingSource['Municipality'] == 'Framingham',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
fram = dict([('Municipality','Framinham'), ('Emergency Fund', values[1]), ('CDBG', values[1]), ('CARES',values[1])])
framingham = pd.DataFrame.from_dict(fram,orient='index').transpose()
framingham

,Municipality,Emergency Fund,CDBG,CARES
0,Framinham,Not specified,Not specified,Not specified


In [29]:
extras = pd.merge(arlington,brookline,how='outer')
extras = pd.merge(extras,boston,how='outer')
extras = pd.merge(extras,cambridge,how='outer')
extras = pd.merge(extras,framingham,how='outer')
extras

,Municipality,CPA,CDBG-CV,Donations,Housing Corporation,Town Sources,Housing Trust,Private Fundraising,Rental Relief Lottery,Federal Stimulus Funds,City Disaster Relief Funds/Donations,Emergency Fund,CDBG,CARES
0,Arlington,"$300,000 CPA (starting late summer/early fall","$400,000 CDBG-CV funds","$115,000 in donations through Arlington Health...","and $100,000 through Housing Corporation of Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brookline,NaN,"$375,000 from CDBG",NaN,NaN,$1.2 million from Town Sources plus Private Fu...,"$100,000 from Brookline Housing Trust","$725,000 from Private Fundraising",NaN,NaN,NaN,NaN,NaN,NaN
2,Boston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Round 1 of a rental relief lottery occurred on...,$5 million in federal stimulus funds.,NaN,NaN,NaN,NaN
3,Cambridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$1.5 million is available to assist cost-burde...,NaN,NaN,NaN
4,Framinham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not specified,Not specified,Not specified


In [30]:
# CARES act funding only
cares = pd.DataFrame()
cares = fundingSource[fundingSource['Funding Source'] == 'CARES Act']
cares = cares.rename(columns={'Amount Allocated':'CARES'})
cares = cares.drop(columns=['Funding Source'])

In [31]:
# trust funding only
trust = pd.DataFrame()
trust = fundingSource[fundingSource['Funding Source'] == 'Trust']
trust = trust.rename(columns={'Amount Allocated':'Trust'})
trust = trust.drop(columns=['Funding Source'])

In [32]:
# only those municipalities for which a funding source was not specified
notSpecified = pd.DataFrame()
notSpecified = fundingSource[fundingSource['Funding Source'] == 'NotSpecified']
notSpecified = notSpecified.rename(columns={'Amount Allocated':'Not Specified'})
notSpecified = notSpecified.drop(columns=['Funding Source'])

In [33]:
# municipalities which received funding from both trust and CPA (only)
trustCPA = pd.DataFrame()
trustCPA = fundingSource[fundingSource['Funding Source'] == 'Trust/CPA']
trustCPA = trustCPA.rename(columns={'Amount Allocated':'Trust/CPA'})
trustCPA = trustCPA.drop(columns=['Funding Source'])

In [34]:
# CPA funding only
cpa = pd.DataFrame()
cpa = fundingSource[fundingSource['Funding Source'] == 'CPA']
cpa = cpa.rename(columns={'Amount Allocated':'CPA'})
cpa = cpa.drop(columns=['Funding Source'])

In [35]:
# ESG-CV funding only
esgCv = pd.DataFrame()
esgCv = fundingSource[fundingSource['Funding Source'] == 'ESG-CV']
esgCv = esgCv.rename(columns={'Amount Allocated':'ESG-CV'})
esgCv = esgCv.drop(columns=['Funding Source'])

In [36]:
# HOME funding only
home = pd.DataFrame()
home = fundingSource[fundingSource['Funding Source']=='HOME']
home = home.rename(columns={'Amount Allocated':'HOME'})
home = home.drop(columns=['Funding Source'])

In [37]:
#CDBG/HOME-TBRA funding
cdbgHome = pd.DataFrame()
cdbgHome = fundingSource[fundingSource['Funding Source']=='CDBG/HOME-TBRA']
cdbgHome = cdbgHome.rename(columns={'Amount Allocated':'CDBG/HOME-TBRA'})
cdbgHome = cdbgHome.drop(columns=['Funding Source'])

In [38]:
# funding from city/mayoral funds
city = pd.DataFrame()
city = fundingSource[fundingSource['Funding Source']=='City funds allocated by Mayor']
city = city.rename(columns={'Amount Allocated':'City Funds Allocated by Mayor'})
city = city.drop(columns=['Funding Source'])

In [39]:
#funding from trust/CDBG-CV
trustCdbgCv = pd.DataFrame()
trustCdbgCv = fundingSource[fundingSource['Funding Source']=="Trust/CDBG-CV"]
trustCdbgCv = trustCdbgCv.rename(columns={'Amount Allocated':"Trust/CDBG-CV"})
trustCdbgCv = trustCdbgCv.drop(columns=['Funding Source'])

In [40]:
# creating base tracker for seeing all data sources broken up
sourcesTrack = pd.DataFrame(columns=['Municipality'])
sourcesTrack['Municipality'] = fundingSource['Municipality']
sourcesTrack['Total Funding'] = fundingSource['Amount Allocated'].values
sourcesTrack['Total Funding'] = sourcesTrack['Total Funding'].apply(cleanTotal)

## Start by merging individual municipalities that needed to be cleaned more
sourcesTrack = pd.merge(sourcesTrack, extras, how='left',on='Municipality')

sourcesTrack = pd.merge(sourcesTrack, cares, how='left',on=['Municipality','CARES'])
sourcesTrack = pd.merge(sourcesTrack, trust, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, notSpecified, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, trustCPA, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, cpa, how='left',on=['Municipality','CPA'])
sourcesTrack = pd.merge(sourcesTrack, esgCv, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, home, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, cdbgHome, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, city, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, trustCdbgCv, how='left',on='Municipality')


sourcesTrack.to_csv('sourceTracker.csv')
sourcesTrack

,Municipality,Total Funding,CPA,CDBG-CV,Donations,Housing Corporation,Town Sources,Housing Trust,Private Fundraising,Rental Relief Lottery,...,CDBG,CARES,Trust,Not Specified,Trust/CPA,ESG-CV,HOME,CDBG/HOME-TBRA,City Funds Allocated by Mayor,Trust/CDBG-CV
0,Acton,55000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amesbury,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Not specified,NaN,NaN,NaN,NaN,NaN,NaN
2,Amherst,250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"$250,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arlington,915000,"$300,000 CPA (starting late summer/early fall","$400,000 CDBG-CV funds","$115,000 in donations through Arlington Health...","and $100,000 through Housing Corporation of Ar...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barnstable County,100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"$100,000",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Weymouth,239773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Williamstown,95700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"$20,700 Trust, $75,000 CPA",NaN,NaN,NaN,NaN,NaN
84,Yarmouth,108000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"$108,000",NaN,NaN,NaN,NaN,NaN
85,"South Berkshire Consortium, see notes*",220000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
